# **Decision Tree Classifier**

The aim of this project is to determine whether the person is a smoker or not

### Step 1: Import the necessary libraries

In [ ]:
#for data manipulation
import numpy as np
import pandas as pd

#for visualization
import matplotlib.pyplot as plt
import seaborn as sns

#for data modelling
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

#for finding out the accuracy of the model
from sklearn import metrics

### Step 2: Load the dataset

In [ ]:
df=pd.read_csv('play_tennis.csv')

### Step 3: Introductory Insights

Obtain introductory information such as shape of the data, number of rows, number of columns, etc.

In [ ]:
df.head(10)

In [ ]:
df.shape

In [ ]:
df.info()

### Step 4: Statistical Insights


In [ ]:
df.describe()

### Step 5: Data Cleaning

Handling outliers, duplicates and missing values

**Missing Values**

In [ ]:
df.isnull().sum()

**Duplicate Values**

In [ ]:
df.duplicated().sum()

### Step 6: Feature Selection

In [ ]:
df=df.drop(['day'],axis=1)

### Step 7: Data Visualisation

### Step 8: Data Modelling

* Train-Test-Split 
* Fit the model to perform predictions
* Using the **Decision Tree Classifier** Algorithm
  - wo
  - re

In [ ]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [ ]:
x

In [ ]:
y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [ ]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion="entropy",  max_depth=4)

# Train Decision Tree Classifer
clf = clf.fit(x_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(x_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
plt.figure(figsize=(10,10))
tree.plot_tree(clf)
plt.show()